In [2]:
#loading libraries
import pandas as pd
import sqlalchemy as sq
from sqlalchemy import create_engine
import numpy as np

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True) 

In [3]:
# loading data
VEQ = pd.read_csv("VEQT.csv")
XEC = pd.read_csv("XEC.csv")
XEF = pd.read_csv("XEF.csv")
XBAL = pd.read_csv("XBAL.csv")

In [4]:
#Adding ETF designation column
VEQ["ETF"] = "VEQ"
XEC["ETF"] = "XEC"
XEF["ETF"] = "XEF"
XBAL["ETF"] = "XBAL"
VEQ.head()

,Date,Price,Open,High,Low,Vol.,Change %,ETF
0,2025-12-12,54.24,54.70,54.72,54.08,378.39K,-0.82%,VEQ
1,2025-12-11,54.69,54.44,54.74,54.30,207.17K,0.24%,VEQ
2,2025-12-10,54.56,54.23,54.63,54.20,284.89K,0.53%,VEQ
3,2025-12-09,54.27,54.23,54.40,54.19,182.68K,-0.06%,VEQ
4,2025-12-08,54.30,54.37,54.40,54.20,286.96K,-0.13%,VEQ


In [5]:
#combining all into dataframe
fin_data = pd.concat([VEQ, XEC, XEF, XBAL], ignore_index=True)

In [6]:
fin_data

,Date,Price,Open,High,Low,Vol.,Change %,ETF
0,2025-12-12,54.24,54.70,54.72,54.08,378.39K,-0.82%,VEQ
1,2025-12-11,54.69,54.44,54.74,54.30,207.17K,0.24%,VEQ
2,2025-12-10,54.56,54.23,54.63,54.20,284.89K,0.53%,VEQ
3,2025-12-09,54.27,54.23,54.40,54.19,182.68K,-0.06%,VEQ
4,2025-12-08,54.30,54.37,54.40,54.20,286.96K,-0.13%,VEQ
...,...,...,...,...,...,...,...,...
595,2025-05-16,30.81,30.72,30.82,30.71,38.56K,0.31%,XBAL
596,2025-05-15,30.72,30.56,30.73,30.56,26.01K,0.57%,XBAL
597,2025-05-14,30.54,30.57,30.57,30.50,41.14K,0.03%,XBAL
598,2025-05-13,30.53,30.52,30.58,30.52,54.25K,0.13%,XBAL


In [7]:
# creating SQL table
fin_data.to_sql("fin_data",engine, index=False,if_exists='replace')

2025-12-14 13:16:29,214 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:29,218 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fin_data")
2025-12-14 13:16:29,219 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,220 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("fin_data")
2025-12-14 13:16:29,220 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,222 INFO sqlalchemy.engine.Engine 
CREATE TABLE fin_data (
	"Date" TEXT, 
	"Price" FLOAT, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Vol." TEXT, 
	"Change %" TEXT, 
	"ETF" TEXT
)


2025-12-14 13:16:29,223 INFO sqlalchemy.engine.Engine [no key 0.00065s] ()
2025-12-14 13:16:29,234 INFO sqlalchemy.engine.Engine INSERT INTO fin_data ("Date", "Price", "Open", "High", "Low", "Vol.", "Change %", "ETF") VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2025-12-14 13:16:29,234 INFO sqlalchemy.engine.Engine [generated in 0.00326s] [('2025-12-12', 54.24, 54.7, 54.72, 54.08, '378.39K', '-0.82%', 'VEQ'), ('2025-12

600

In [8]:
fin_data_table_df = pd.read_sql_table("fin_data", engine)

2025-12-14 13:16:29,253 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:29,254 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fin_data")
2025-12-14 13:16:29,255 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,256 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-12-14 13:16:29,257 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,258 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-12-14 13:16:29,259 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,260 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("fin_data")
2025-12-14 13:16:29,260 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,263 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND 

In [9]:
fin_data_table_df.head()

,Date,Price,Open,High,Low,Vol.,Change %,ETF
0,2025-12-12,54.24,54.70,54.72,54.08,378.39K,-0.82%,VEQ
1,2025-12-11,54.69,54.44,54.74,54.30,207.17K,0.24%,VEQ
2,2025-12-10,54.56,54.23,54.63,54.20,284.89K,0.53%,VEQ
3,2025-12-09,54.27,54.23,54.40,54.19,182.68K,-0.06%,VEQ
4,2025-12-08,54.30,54.37,54.40,54.20,286.96K,-0.13%,VEQ


In [10]:
sql_create_view = """
DROP VIEW IF EXISTS v_clean_prices;

CREATE VIEW v_clean_prices AS
WITH base AS (
  SELECT
    dt,
    etf,
    "Change %" AS change_raw,
    CASE
      WHEN "Change %" IS NULL OR TRIM("Change %") = '' THEN NULL
      ELSE CAST(REPLACE(REPLACE(TRIM("Change %"), '%', ''), ',', '') AS REAL) / 100.0
    END AS change_pct
  FROM fin_data
)
SELECT *
FROM base
WHERE dt IS NOT NULL
  AND etf IS NOT NULL
  AND TRIM(etf) <> '';
"""

In [11]:
# check for Null values in data
Null_query = pd.read_sql_query(
    """SELECT ETF,
  SUM(CASE WHEN Price IS NULL OR Price <= 0 THEN 1 ELSE 0 END) AS bad_price_rows,
  SUM(CASE WHEN Open  IS NULL OR Open  <= 0 THEN 1 ELSE 0 END) AS bad_open_rows,
  SUM(CASE WHEN High  IS NULL OR High  <= 0 THEN 1 ELSE 0 END) AS bad_high_rows,
  SUM(CASE WHEN Low   IS NULL OR Low   <= 0 THEN 1 ELSE 0 END) AS bad_low_rows,
  SUM(CASE WHEN High < Low THEN 1 ELSE 0 END) AS high_lt_low_rows
FROM fin_data
GROUP BY ETF
ORDER BY ETF

    ;""",
    engine
)
Null_query

2025-12-14 13:16:29,346 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:29,347 INFO sqlalchemy.engine.Engine SELECT ETF,
  SUM(CASE WHEN Price IS NULL OR Price <= 0 THEN 1 ELSE 0 END) AS bad_price_rows,
  SUM(CASE WHEN Open  IS NULL OR Open  <= 0 THEN 1 ELSE 0 END) AS bad_open_rows,
  SUM(CASE WHEN High  IS NULL OR High  <= 0 THEN 1 ELSE 0 END) AS bad_high_rows,
  SUM(CASE WHEN Low   IS NULL OR Low   <= 0 THEN 1 ELSE 0 END) AS bad_low_rows,
  SUM(CASE WHEN High < Low THEN 1 ELSE 0 END) AS high_lt_low_rows
FROM fin_data
GROUP BY ETF
ORDER BY ETF

    ;
2025-12-14 13:16:29,348 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:29,350 INFO sqlalchemy.engine.Engine ROLLBACK


,ETF,bad_price_rows,bad_open_rows,bad_high_rows,bad_low_rows,high_lt_low_rows
0,VEQ,0,0,0,0,0
1,XBAL,0,0,0,0,0
2,XEC,0,0,0,0,0
3,XEF,0,0,0,0,0


No Null Values

In [19]:
# outlier / duplicate check

duplicatecheck = pd.read_sql_query(
    """
    SELECT
      ETF,
      Date,
      COUNT(*) AS n
    FROM fin_data
    GROUP BY ETF, Date
    HAVING COUNT(*) > 1
    ORDER BY ETF, Date;
    """,
    engine
)
duplicatecheck

2025-12-14 13:16:32,501 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:32,503 INFO sqlalchemy.engine.Engine 
    SELECT
      ETF,
      Date,
      COUNT(*) AS n
    FROM fin_data
    GROUP BY ETF, Date
    HAVING COUNT(*) > 1
    ORDER BY ETF, Date;
    
2025-12-14 13:16:32,503 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:32,506 INFO sqlalchemy.engine.Engine ROLLBACK


,ETF,Date,n


No Duplicates since table is empty

## Daily Returns

### Queries 

In [27]:
fund_daily_performance = pd.read_sql_query("""
WITH ordered AS (
  SELECT
    ETF,
    Date,
    Price,
    LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) AS prev_price
  FROM fin_data
)
SELECT
  ETF,
  Date,
  Price,
  prev_price,
  CASE
    WHEN prev_price IS NULL OR prev_price <= 0 THEN NULL
    ELSE (Price / prev_price) - 1.0
  END AS daily_return,
  CASE
    WHEN prev_price IS NULL OR prev_price <= 0 OR Price <= 0 THEN NULL
    ELSE ln(Price / prev_price)
  END AS daily_log_return
FROM ordered
ORDER BY ETF, Date;

""",
    engine
)

fund_daily_performance

2025-12-14 13:16:35,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:35,035 INFO sqlalchemy.engine.Engine 
WITH ordered AS (
  SELECT
    ETF,
    Date,
    Price,
    LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) AS prev_price
  FROM fin_data
)
SELECT
  ETF,
  Date,
  Price,
  prev_price,
  CASE
    WHEN prev_price IS NULL OR prev_price <= 0 THEN NULL
    ELSE (Price / prev_price) - 1.0
  END AS daily_return,
  CASE
    WHEN prev_price IS NULL OR prev_price <= 0 OR Price <= 0 THEN NULL
    ELSE ln(Price / prev_price)
  END AS daily_log_return
FROM ordered
ORDER BY ETF, Date;


2025-12-14 13:16:35,036 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:35,040 INFO sqlalchemy.engine.Engine ROLLBACK


,ETF,Date,Price,prev_price,daily_return,daily_log_return
0,VEQ,2025-05-12,46.09,NaN,NaN,NaN
1,VEQ,2025-05-13,46.17,46.09,0.001736,0.001734
2,VEQ,2025-05-14,46.30,46.17,0.002816,0.002812
3,VEQ,2025-05-15,46.56,46.30,0.005616,0.005600
4,VEQ,2025-05-16,46.83,46.56,0.005799,0.005782
...,...,...,...,...,...,...
595,XEF,2025-12-08,46.08,46.13,-0.001084,-0.001084
596,XEF,2025-12-09,46.02,46.08,-0.001302,-0.001303
597,XEF,2025-12-10,46.33,46.02,0.006736,0.006714
598,XEF,2025-12-11,46.50,46.33,0.003669,0.003663


In [28]:
rolling_returns = pd.read_sql_query("""
WITH dr AS (
  SELECT
    ETF,
    Date,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS daily_return
  FROM fin_data
),
rr AS (
  SELECT
    ETF,
    Date,
    daily_return,
    exp(SUM(CASE WHEN daily_return IS NULL THEN 0 ELSE ln(1.0 + daily_return) END)
        OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW)) - 1.0 AS roll_return_20d,
    exp(SUM(CASE WHEN daily_return IS NULL THEN 0 ELSE ln(1.0 + daily_return) END)
        OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW)) - 1.0 AS roll_return_60d
  FROM dr
)
SELECT *
FROM rr
ORDER BY ETF, Date;
""", engine)
rolling_returns

2025-12-14 13:16:35,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:35,058 INFO sqlalchemy.engine.Engine 
WITH dr AS (
  SELECT
    ETF,
    Date,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS daily_return
  FROM fin_data
),
rr AS (
  SELECT
    ETF,
    Date,
    daily_return,
    exp(SUM(CASE WHEN daily_return IS NULL THEN 0 ELSE ln(1.0 + daily_return) END)
        OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW)) - 1.0 AS roll_return_20d,
    exp(SUM(CASE WHEN daily_return IS NULL THEN 0 ELSE ln(1.0 + daily_return) END)
        OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW)) - 1.0 AS roll_return_60d
  FROM dr
)
SELECT *
FROM rr
ORDER BY ETF, Date;

2025-12-14 13:16:35,058 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:35,065 INFO sqlalchemy.engine.Engine ROLLBAC

,ETF,Date,daily_return,roll_return_20d,roll_return_60d
0,VEQ,2025-05-12,NaN,0.000000,0.000000
1,VEQ,2025-05-13,0.001736,0.001736,0.001736
2,VEQ,2025-05-14,0.002816,0.004556,0.004556
3,VEQ,2025-05-15,0.005616,0.010197,0.010197
4,VEQ,2025-05-16,0.005799,0.016056,0.016056
...,...,...,...,...,...
595,XEF,2025-12-08,-0.001084,-0.007324,0.024911
596,XEF,2025-12-09,-0.001302,-0.014983,0.022212
597,XEF,2025-12-10,0.006736,-0.012995,0.032538
598,XEF,2025-12-11,0.003669,-0.002574,0.039571


### Volatility Breakdown

In [30]:
volatility_peretf = pd.read_sql_query("""
WITH dr AS (
  SELECT
    ETF,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS r
  FROM fin_data
)
SELECT
  ETF,
  COUNT(r) AS n_days,
  AVG(r) AS avg_daily_return,
  sqrt(AVG(r*r) - AVG(r)*AVG(r)) * sqrt(252.0) AS vol_annualized
FROM dr
WHERE r IS NOT NULL
GROUP BY ETF
ORDER BY ETF;

""", engine)

volatility_peretf

2025-12-14 13:16:35,094 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:35,095 INFO sqlalchemy.engine.Engine 
WITH dr AS (
  SELECT
    ETF,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS r
  FROM fin_data
)
SELECT
  ETF,
  COUNT(r) AS n_days,
  AVG(r) AS avg_daily_return,
  sqrt(AVG(r*r) - AVG(r)*AVG(r)) * sqrt(252.0) AS vol_annualized
FROM dr
WHERE r IS NOT NULL
GROUP BY ETF
ORDER BY ETF;


2025-12-14 13:16:35,096 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-14 13:16:35,098 INFO sqlalchemy.engine.Engine ROLLBACK


,ETF,n_days,avg_daily_return,vol_annualized
0,VEQ,149,0.001112,0.097150
1,XBAL,149,0.000625,0.066966
2,XEC,149,0.001004,0.128234
3,XEF,149,0.000684,0.110881


In [31]:
volatility_perdate = pd.read_sql_query(""" 
WITH dr AS (
  SELECT
    ETF,
    Date,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS r
  FROM fin_data
),
roll AS (
  SELECT
    ETF,
    Date,
    r,
    -- rolling mean
    AVG(r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW) AS m20,
    AVG(r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS m60,
    -- rolling mean of squares
    AVG(r*r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW) AS msq20,
    AVG(r*r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS msq60
  FROM dr
),
out AS (
  SELECT
    ETF,
    Date,
    r AS daily_return,
    CASE WHEN m20 IS NULL THEN NULL ELSE sqrt(msq20 - m20*m20) * sqrt(252.0) END AS vol_20d_annualized,
    CASE WHEN m60 IS NULL THEN NULL ELSE sqrt(msq60 - m60*m60) * sqrt(252.0) END AS vol_60d_annualized
  FROM roll
)
SELECT *
FROM out
ORDER BY ETF, Date;

""", engine)
volatility_perdate

2025-12-14 13:16:35,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:16:35,117 INFO sqlalchemy.engine.Engine  
WITH dr AS (
  SELECT
    ETF,
    Date,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS r
  FROM fin_data
),
roll AS (
  SELECT
    ETF,
    Date,
    r,
    -- rolling mean
    AVG(r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW) AS m20,
    AVG(r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS m60,
    -- rolling mean of squares
    AVG(r*r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 19 PRECEDING AND CURRENT ROW) AS msq20,
    AVG(r*r) OVER (PARTITION BY ETF ORDER BY Date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS msq60
  FROM dr
),
out AS (
  SELECT
    ETF,
    Date,
    r AS daily_return,
    CASE WHEN m20 IS NULL THEN NULL ELSE sqrt(msq20 - m20*m20) * 

,ETF,Date,daily_return,vol_20d_annualized,vol_60d_annualized
0,VEQ,2025-05-12,NaN,NaN,NaN
1,VEQ,2025-05-13,0.001736,0.000000,0.000000
2,VEQ,2025-05-14,0.002816,0.008572,0.008572
3,VEQ,2025-05-15,0.005616,0.025955,0.025955
4,VEQ,2025-05-16,0.005799,0.027922,0.027922
...,...,...,...,...,...
595,XEF,2025-12-08,-0.001084,0.119693,0.108012
596,XEF,2025-12-09,-0.001302,0.117121,0.108052
597,XEF,2025-12-10,0.006736,0.118508,0.108532
598,XEF,2025-12-11,0.003669,0.117151,0.108443


In [42]:
Sharpe_RatioQ = pd.read_sql_query("""
WITH daily_returns AS (
  SELECT
    ETF,
    Date,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS daily_return
  FROM fin_data
),
stats AS (
  SELECT
    ETF,
    COUNT(daily_return) AS n_days,
    AVG(daily_return) AS avg_daily_return,
    AVG(daily_return * daily_return) AS avg_sq_daily_return
  FROM daily_returns
  WHERE daily_return IS NOT NULL
  GROUP BY ETF
)
SELECT
  ETF,
  n_days,
  avg_daily_return * 252.0 AS annualized_return,
  sqrt(avg_sq_daily_return - avg_daily_return * avg_daily_return) * sqrt(252.0) AS annualized_volatility,
  CASE
    WHEN (avg_sq_daily_return - avg_daily_return * avg_daily_return) <= 0 THEN NULL
    ELSE (avg_daily_return * 252.0) /
         (sqrt(avg_sq_daily_return - avg_daily_return * avg_daily_return) * sqrt(252.0))
  END AS sharpe_ratio
FROM stats
ORDER BY sharpe_ratio DESC;

""",engine)
Sharpe_RatioQ

2025-12-14 13:25:15,397 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-14 13:25:15,398 INFO sqlalchemy.engine.Engine 
WITH daily_returns AS (
  SELECT
    ETF,
    Date,
    CASE
      WHEN LAG(Price) OVER (PARTITION BY ETF ORDER BY Date) IS NULL THEN NULL
      ELSE (Price / LAG(Price) OVER (PARTITION BY ETF ORDER BY Date)) - 1.0
    END AS daily_return
  FROM fin_data
),
stats AS (
  SELECT
    ETF,
    COUNT(daily_return) AS n_days,
    AVG(daily_return) AS avg_daily_return,
    AVG(daily_return * daily_return) AS avg_sq_daily_return
  FROM daily_returns
  WHERE daily_return IS NOT NULL
  GROUP BY ETF
)
SELECT
  ETF,
  n_days,
  avg_daily_return * 252.0 AS annualized_return,
  sqrt(avg_sq_daily_return - avg_daily_return * avg_daily_return) * sqrt(252.0) AS annualized_volatility,
  CASE
    WHEN (avg_sq_daily_return - avg_daily_return * avg_daily_return) <= 0 THEN NULL
    ELSE (avg_daily_return * 252.0) /
         (sqrt(avg_sq_daily_return - avg_daily_return * avg_daily_retu

,ETF,n_days,annualized_return,annualized_volatility,sharpe_ratio
0,VEQ,149,0.280252,0.097150,2.884730
1,XBAL,149,0.157476,0.066966,2.351583
2,XEC,149,0.253083,0.128234,1.973605
3,XEF,149,0.172417,0.110881,1.554969


Source: https://www.codeproject.com/articles/Sharpe-Ratio-and-Sortino-Ratio-for-a-Portfolio-in-#comments-section

## CSV Outputs

In [ ]:
volatility_peretf.to_csv("volatility_peretf.csv")
volatility_perdate.to_csv("volatility_perdate.csv")
fund_daily_performance.to_csv("fund_daily_performance.csv")
Sharpe_RatioQ.to_csv("Sharpe_RatioQ.csv")